### O-AUTH FOR GEMINI FINE TUNING ACCESS

In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET_2'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=822844456417-tmrthlu4g15lhjjmusvs0umcqt26a3vm.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=3sDMfL1UIgXmVop30dY09pqoeoPEBK&access_type=offline&code_challenge=O12NCgeZGQSFH-PV5shjOsgix0bmzWuSXVvSK9CtoK4&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=3sDMfL1UIgXmVop30dY09pqoeoPEBK&code=4/0ATx3LY6ajTA-W1Lpx6oxcO8rpAOqiR-rSrOPLgpWWqtIOXXd5CpugMs24GhsDeybpSltgQ&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generati

In [ ]:
import google.generativeai as genai

#### Fine Tuned Engineering Type Classifier Model

In [ ]:
model = genai.GenerativeModel(model_name='tunedModels/classifierpecimodel-lqys2wyq40t7')

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_excel('check2.xlsx')

#### Dataframe with concatenated list

In [ ]:
df.head()

,Unnamed: 0,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order
0,0,SA24M005,Contractual Milestones,NaN,1,-1,0
1,1,SA24M005,Portion- I,NaN,2,-1,0
2,2,SA24M005,Complete Mobilization and Commence the work- P...,NaN,0,Contractual Milestones Portion- I Comp...,1
3,3,SA24M005,Technical Completion of the Project ( TCC )- P...,NaN,0,Contractual Milestones Portion- I Tech...,2
4,4,SA24M005,Preliminary Acceptance of the Project ( PAC )-...,NaN,0,Contractual Milestones Portion- I Prel...,3


## Gemini Fine Tuning Model Usage

In [ ]:
import time
import random
import string
from datetime import datetime
from google.api_core.exceptions import InternalServerError

predicted_cat = []
cnt = -1
cnt_req = 0
name = ''

# Total number of requests you want to process
total_requests = len(df['Concat'])

def generate_code(length=4):
    letters = string.ascii_letters
    return 'X'

# Function to process requests with delay and retry mechanism
def process_requests(rate_limit, interval, max_retries=3, retry_delay=5):
    global cnt, cnt_req, name
    for i in df['Concat']:
        cnt += 1

        if i != '-1' and df['Order'][cnt] == 1:
            cnt_req += 1
            retries = 0
            while retries < max_retries:
                try:
                    if cnt_req % rate_limit == 0 and cnt_req != 0:
                        print(f"Sleeping .. {datetime.now()}:")
                        print(cnt_req)
                        time.sleep(interval)

                    result = model.generate_content(i)
                    name = result.text
                    break
                except ValueError:
                    print(f"Error processing request {cnt_req} and row number {cnt}. Generating code.")
                    name = generate_code()
                    break
                except InternalServerError:
                    print(f"InternalServerError for request {cnt}. Retrying in {retry_delay} seconds.")
                    time.sleep(retry_delay)
                    retries += 1
                    if retries == max_retries:
                        print(f"Max retries reached for request {cnt}. Generating random code.")
                        name = generate_code()
            predicted_cat.append(name)
            print(f"Processed request {cnt} at {datetime.now()}")
            print(name, i)

        elif i != '-1' and (cnt == 7164 or df['Order'][cnt + 1] == 0):
            for _ in range(df['Order'][cnt] - 1):
                predicted_cat.append(name)

        if i == '-1':
            predicted_cat.append(' ')

# Rate limit of 16 requests per minute
rate_limit = 16
# Sleep interval of 61 seconds (1 minute and 1 second)
interval = 61

process_requests(rate_limit, interval)


Processed request 2 at 2024-06-26 10:37:03.871098
Category: Contracts    Contractual Milestones      Portion- I Complete Mobilization and Commence the work- Portion I
Processed request 7 at 2024-06-26 10:37:04.888480
Category: Contracts    Contractual Milestones      Portion- II Complete Mobilization and Commence the work- Portion II
Processed request 12 at 2024-06-26 10:37:05.991672
Category: Contracts    Internal Milestones Contract Award Date
Processed request 21 at 2024-06-26 10:37:07.066685
Category: Engineering    Project Portion- I (Construction of North Qassim 380kV BSP & Installation of New Reactors      Main Station- North Qassim 380kV BSP (9061 BSP)        Preliminary Works          General Works Kick off Meeting with Project Team
Processed request 37 at 2024-06-26 10:37:08.136862
Category: Engineering    Project Portion- I (Construction of North Qassim 380kV BSP & Installation of New Reactors      Main Station- North Qassim 380kV BSP (9061 BSP)        Preliminary Works     

### Saving Part of Code Generation

In [ ]:
df['Classification'] = predicted_cat
df.to_excel('Partial2.xlsx')

In [ ]:
from google.colab import files
files.download('Partial2.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Accounting for duplicate codes generated

In [80]:
import numpy as np
import pandas as pd
df3 = pd.read_excel('Generated-final.xlsx')

In [81]:
df3.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order,Middle Part,Classification,Last Four,Code Final
0,0,0,0,SA24M005,Contractual Milestones,NaN,1,-1,0,,,X,
1,1,1,1,SA24M005,Portion- I,NaN,2,-1,0,,,X,
2,2,2,2,SA24M005,Complete Mobilization and Commence the work- P...,NaN,0,Contractual Milestones Portion- I Comp...,1,CM00MW,L000,00L1,L000CM00MW00L1
3,3,3,3,SA24M005,Technical Completion of the Project ( TCC )- P...,NaN,0,Contractual Milestones Portion- I Tech...,2,CM00MW,L000,00L2,L000CM00MW00L2
4,4,4,4,SA24M005,Preliminary Acceptance of the Project ( PAC )-...,NaN,0,Contractual Milestones Portion- I Prel...,3,CM00MW,L000,00L3,L000CM00MW00L3


In [82]:
# Group by 'Code Final' and count the number of 'Order' == 1
grouped = df3[df3['Order'] == 1].groupby('Code Final').size()

# Filter the groups that have more than one 'Order' == 1
duplicates = grouped[grouped > 1]

print("Codes with more than one 'Order' == 1:")
duplicates

Codes with more than one 'Order' == 1:


Code Final
C00000LR00C1           2
C000A0RMR00C1          2
C000C0F0P00C1          2
C000C0IPP00FW0000C1    2
C000C0IPP0SUB00C1      2
                      ..
P000MHVAC00P1          4
P000OPGW00P1           2
P000PDMS00P1           2
P000PR0SCM00P1         4
P000SRGA00P1           3
Length: 105, dtype: int64

In [83]:
cnt_index_as = -1
checker = []
import random
import string
def generate_code(length=4):
    letters = string.ascii_letters
    return '#' + ''.join(random.choice(letters) for _ in range(length))
for i in df3['Code Final']:
  cnt_index_as +=1
  if i in duplicates:
    if i not in checker:
      checker.append(i)
    rand_name = generate_code()
    df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
    print(df3['Code Final'][cnt_index_as])

<ipython-input-83-b9c84e3ba645>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
<ipython-input-83-b9c84e3ba645>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
<ipython-input-83-b9c84e3ba645>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
<ipytho

E000#IomO00E1
E000#sCEz00E1
E000#Oxzl00E1
E000#VlZt00E1
E000#ofee00E1
E000#sqAx00E1
E000#jbjk00E1
E000#bvDq00E1
E000#dPWH00E1
E000#bjmW00E1
E000#TmRQ00E1
E000#dARI00E1
E000#fYlJ00E1
E000#BTBZ00E1
E000#SVxl00E1
E000#PfOn00E1
E000#pVbn00E1
E000#hQMh00E1
E000#vMqh00E1
E000#AhVI00E1
E000#BMJt00E1
E000#SZZJ00E1
E000#QfDo00E1
E000#sVzQ00E1
E000#aVVH00E1
E000#RVjp00E1
E000#OeFp00E1
E000#iLzP00E1
E000#Ztdn00E1
E000#wADi00E1
E000#ciMI00E1
E000#pKcD00E1
E000#ziEx00E1
E000#KEkU00E1
E000#Nwzs00E1
E000#atis00E1
E000#fZan00E1
E000#SaqZ00E1
E000#LVfm00E1
E000#Smkf00E1
E000#doEr00E1
E000#LonY00E1
E000#pDTt00E1
E000#oFkQ00E1
E000#Dlnl00E1
E000#GUck00E1
E000#lHzj00E1
E000#NExf00E1
E000#fZmf00E1
E000#YoFV00E1
E000#KDLT00E1
E000#PJoZ00E1
E000#DtML00E1
E000#TmrQ00E1
E000#bZWD00E1
E000#roPP00E1
E000#nORL00E1
E000#tPMk00E1
E000#VbNX00E1
E000#TAnf00E1
E000#MTWj00E1
E000#fYpj00E1
E000#osnv00E1
E000#hPjS00E1
E000#ULOj00E1
E000#dFgA00E1
E000#WEZI00E1
E000#gsLy00E1
E000#hdPi00E1
E000#bLMz00E1
E000#jDHG00E1
E000#b

<ipython-input-83-b9c84e3ba645>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
<ipython-input-83-b9c84e3ba645>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
<ipython-input-83-b9c84e3ba645>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Code Final'][cnt_index_as] = i[0:4] + rand_name + i[-4:]
<ipytho

In [84]:
len(checker)

105

In [85]:
# Group by 'Code Final' and count the number of 'Order' == 1
grouped = df3[df3['Order'] == 1].groupby('Code Final').size()

# Filter the groups that have more than one 'Order' == 1
duplicates = grouped[grouped > 1]

print("Codes with more than one 'Order' == 1:")
duplicates

Codes with more than one 'Order' == 1:


Series([], dtype: int64)

In [86]:
cnts = -1
while cnts < len(df3) - 1:
    cnts += 1
    if df3['Order'][cnts] == 1:
        x = df3['Code Final'][cnts][:-4]
        if cnts + 1 < len(df3) and df3['Code Final'][cnts][:-4] != df3['Code Final'][cnts + 1][:-4]:
            cnt = cnts + 1
            while cnt < len(df3) and df3['Order'][cnt] != 1:
                df3.at[cnt, 'Code Final'] = x + df3['Code Final'][cnt][-4:]
                cnt += 1
            cnts = cnt - 1  # adjust cnts to continue from the next 'Order' == 1


In [87]:
# Group by 'Code Final' and count the number of 'Order' == 1
grouped = df3[df3['Order'] == 1].groupby('Code Final').size()

# Filter the groups that have more than one 'Order' == 1
duplicates = grouped[grouped > 1]

print("Codes with more than one 'Order' == 1:")
print(duplicates)

Codes with more than one 'Order' == 1:
Series([], dtype: int64)


In [90]:
df3.to_excel('GeneratedCodes-Final-KSADataLake-SA24M005.xlsx')

In [91]:
from google.colab import files
files.download('GeneratedCodes-Final-KSADataLake-SA24M005.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>